Simplifications made:
* jail: assumes player rolls to get out of jail and only pays to get out after 3 turns OR with p = 0.5
* get out of jail free card is played with p = 0.5 if posessed
* assumes number of goes per game is normally distributed with mean = 30 and SD = 5 (discretised normal since the number of goes needs to be integer value)

now accounts for this:
* if a player draws a chance card where they need to move a certain number of spaces, they don't take action on the space they move to (this is fairly accurate since there are only 3 spaces that are 3 spaces back from chance - 2 properties and 1 community chest)
* community chest and chance cards would not be random, instead there would be a randomised set order

card lists taken from: https://www.monopolyland.com/list-monopoly-chance-community-chest-cards/

In [1]:
import random
import numpy as np
import plotly.express as px
import pandas as pd
import scipy.stats as sts

# Define the Monopoly board with London-themed property names and their corresponding colors

monopoly_board_london = [
    ("Go", "lightgray", None, None, None, None, None, None, None, None),
    ("Old Kent Road", "brown", 2, 10, 30, 90, 160, 250, 60, 50),
    ("Community Chest 1", "lightgray", None, None, None, None, None, None, None, None),
    ("Whitechapel Road", "brown", 4, 20, 60, 180, 320, 450, 80, 50),
    ("Income Tax", "lightgray", None, None, None, None, None, None, None, None),
    ("King's Cross Station", "black", None, None, None, None, None, None, 200, None),
    ("The Angel, Islington", "lightblue", 6, 30, 90, 270, 400, 550, 100, 50),
    ("Chance 1", "lightgray", None, None, None, None, None, None, None, None),
    ("Euston Road", "lightblue", 6, 30, 90, 270, 400, 550, 100, 50),
    ("Pentonville Road", "lightblue", 8, 40, 100, 300, 450, 600, 120, 50),
    ("Jail", "lightgray", None, None, None, None, None, None, None, None),
    ("Pall Mall", "pink", 10, 50, 150, 450, 625, 750, 140, 100),
    ("Electric Company", "lightgray", None, None, None, None, None, None, 150, None),
    ("Whitehall", "pink", 10, 50, 150, 450, 625, 750, 140, 100),
    ("Northumberland Avenue", "pink", 12, 60, 180, 500, 700, 900, 160, 100),
    ("Marylebone Station", "black", None, None, None, None, None, None, 200, None),
    ("Bow Street", "orange", 14, 70, 200, 550, 750, 950, 180, 100),
    ("Community Chest 2", "lightgray", None, None, None, None, None, None, None, None),
    ("Marlborough Street", "orange", 14, 70, 200, 550, 750, 950, 180, 100),
    ("Vine Street", "orange", 16, 80, 220, 600, 800, 1000, 200, 100),
    ("Free Parking", "lightgray", None, None, None, None, None, None, None, None),
    ("Strand", "red", 18, 90, 250, 700, 875, 1050, 220, 150),
    ("Chance 2", "lightgray", None, None, None, None, None, None, None, None),
    ("Fleet Street", "red", 18, 90, 250, 700, 875, 1050, 220, 150),
    ("Trafalgar Square", "red", 20, 100, 300, 750, 925, 1100, 240, 150),
    ("Fenchurch St. Station", "black", None, None, None, None, None, None, 200, None),
    ("Leicester Square", "yellow", 22, 110, 330, 800, 975, 1150, 260, 150),
    ("Coventry Street", "yellow", 22, 110, 330, 800, 975, 1150, 260, 150),
    ("Water Works", "lightgray", None, None, None, None, None, None, 150, None),
    ("Piccadilly", "yellow", 24, 120, 360, 850, 1025, 1200, 280, 150),
    ("Go to Jail", "lightgray", None, None, None, None, None, None, None, None),
    ("Regent Street", "green", 26, 130, 390, 900, 1100, 1275, 300, 200),
    ("Oxford Street", "green", 26, 130, 390, 900, 1100, 1275, 300, 200),
    ("Community Chest 3", "lightgray", None, None, None, None, None, None, None, None),
    ("Bond Street", "green", 28, 150, 450, 1000, 1200, 1400, 320, 200),
    ("Liverpool St. Station", "black", None, None, None, None, None, None, 200, None),
    ("Chance 3", "lightgray", None, None, None, None, None, None, None, None),
    ("Park Lane", "darkblue", 35, 175, 500, 1100, 1300, 1500, 350, 200),
    ("Super Tax", "lightgray", None, None, None, None, None, None, None, None),
    ("Mayfair", "darkblue", 50, 200, 600, 1400, 1700, 2000, 400, 200)
]

chance_cards = [
    "Advance to Go.",
    "Advance to Trafalgar Square. If you pass Go, collect £200.",
    "Advance to Mayfair.",
    "Advance to Pall Mall. If you pass Go, collect £200.",
    "Advance to the nearest Station. If unowned, you may buy it from the Bank. If owned, pay the owner twice the rental to which they are otherwise entitled.",
    "Advance to the nearest Station. If unowned, you may buy it from the Bank. If owned, pay the owner twice the rental to which they are otherwise entitled.",
    "Advance token to nearest Utility. If unowned, you may buy it from the Bank. If owned, throw dice and pay owner a total ten times the amount thrown.",
    "Bank pays you a dividend of £50.",
    "Get Out of Jail Free.",
    "Go back 3 Spaces.",
    "Go to Jail. Go directly to Jail, do not pass Go, do not collect £200.",
    "Make general repairs on all your property. For each house pay £25. For each hotel pay £100",
    "Speeding fine £15.",
    "Take a trip to King's Cross Station. If you pass Go, collect £200.",
    "You have been elected Chairman of the Board. Pay each player £50.",
    "Your building loan matures. Collect £150."
]

community_chest_cards = [
    "Advance to Go.",
    "Bank error in your favor. Collect £200.",
    "Doctor’s fee. Pay £50.",
    "From sale of stock you get £50.",
    "Get Out of Jail Free.",
    "Go to Jail. Go directly to jail, do not pass Go, do not collect £200.",
    "Holiday fund matures. Receive £100.",
    "Income tax refund. Collect £20.",
    "It is your birthday. Collect £10 from every player.",
    "Life insurance matures. Collect £100.",
    "Pay hospital fees of £100.",
    "Pay school fees of £50.",
    "Receive £25 consultancy fee.",
    "You are assessed for street repairs. £40 per house. £115 per hotel.",
    "You have won second prize in a beauty contest. Collect £10.",
    "You inherit £100."
]

# Create a DataFrame to store property names and probabilities
property_names = [prop[0] for prop in monopoly_board_london]
property_colors = [prop[1] for prop in monopoly_board_london]
property_cost = [prop[8] for prop in monopoly_board_london]
property_house_cost = [prop[9] for prop in monopoly_board_london]
property_rent = [prop[2] for prop in monopoly_board_london]
property_rent_doubled = [(prop[2] * 2) if prop[2] is not None else None for prop in monopoly_board_london]
property_rent_1 = [prop[3] for prop in monopoly_board_london]
property_rent_2 = [prop[4] for prop in monopoly_board_london]
property_rent_3 = [prop[5] for prop in monopoly_board_london]
property_rent_4 = [prop[6] for prop in monopoly_board_london]
property_rent_hotel = [prop[7] for prop in monopoly_board_london]
df = pd.DataFrame({'Property': property_names, 'Color': property_colors, 'Purchase Price': property_cost, 'House Price': property_house_cost,
                   'Rent': property_rent, 'Rent Doubled': property_rent_doubled,
                   'Rent 1 House': property_rent_1, 'Rent 2 Houses': property_rent_2, 'Rent 3 Houses': property_rent_3,
                   'Rent 4 Houses': property_rent_4, 'Rent Hotel': property_rent_hotel, 'Probability': 0})

# Create dictionaries to map property names to their indices and colors
property_indices = {property_name: idx for idx, property_name in enumerate(property_names)}
color_dict = {property_name: color for property_name, color in zip(property_names, property_colors)}

# Define the number of simulation rounds and the starting position
num_simulations = 10000
num_moves_mean = 30 # change to the average number of gos for a monopoly game per player
num_moves_sd = 5
start_position = 0
num_spaces = len(monopoly_board_london)
num_chance_cards = len(chance_cards)
num_community_chest_cards = len(community_chest_cards)

# initialise probabilities
p_paying_jail = 0.5
p_using_get_out_jail = 0.5

# Initialise a list to keep track of the frequency of landing on each space
space_freq = [0] * num_spaces

# Simulate the player's random walk and update frequencies
for _ in range(num_simulations):
    position = start_position
    jail_rounds = 0
    in_jail = False
    get_out_jail = False

    # randomise order of chance and community chest cards and set current top card
    random.shuffle(chance_cards)
    random.shuffle(community_chest_cards)
    top_chance_card_idx = 0
    top_community_chest_card_idx = 0

    # randomise number of moves using normal distribution
    num_moves = int(sts.norm(loc = num_moves_mean, scale = num_moves_sd).rvs(size = 1)[0])

    for _ in range(num_moves):
        turn = True
        doubles_count = 0

        while turn:
            # Roll two dice
            die1 = random.randint(1, 6)
            die2 = random.randint(1, 6)

            # Check for doubles
            if die1 == die2:
                doubles_count += 1
            else:
                turn = False

            # Check if player is rolling to get out of jail or if they have a get out of jail card or if they will pay to get out
            if in_jail:
                if get_out_jail and random.random() < p_using_get_out_jail:
                    in_jail = False
                    get_out_jail = False
                elif random.random() < p_paying_jail:
                    in_jail = False
                elif die1 == die2:
                    in_jail = False
                elif jail_rounds == 3:
                    in_jail = False
                else:
                    jail_rounds += 1
                    continue

            position = (position + die1 + die2) % num_spaces

            # If the player rolls three doubles in a row, go to jail
            if doubles_count == 3:
                # add current position before moving
                space_freq[position] += 1

                position = property_indices["Jail"]
                in_jail = True
                turn = False

            # If the player lands on jail, their turn is over
            elif property_names[position] == "Jail":
                turn = False

            # If the player lands on "Go to Jail", they move to jail and their turn is over
            elif property_names[position] == "Go to Jail":
                # add current position before moving
                space_freq[position] += 1

                position = property_indices["Jail"]
                in_jail = True
                turn = False

            # If the players lands on a chance position
            elif property_names[position][:6] == "Chance":
                # Simulate drawing a Chance card
                chance_card = chance_cards[top_chance_card_idx]
                top_chance_card_idx = (top_chance_card_idx + 1) % num_chance_cards

                if "Advance to the nearest Station" in chance_card:
                    # add current position before moving
                    space_freq[position] += 1

                    pos_king = property_indices["King's Cross Station"]
                    pos_mar = property_indices["Marylebone Station"]
                    pos_fen = property_indices["Fenchurch St. Station"]
                    pos_liv = property_indices["Liverpool St. Station"]

                    if pos_king < position: dist_to_king = num_spaces + (pos_king - position)
                    else: dist_to_king = pos_king - position
                    if pos_mar < position: dist_to_mar = num_spaces + (pos_mar - position)
                    else: dist_to_mar = pos_mar - position
                    if pos_fen < position: dist_to_fen = num_spaces + (pos_fen - position)
                    else: dist_to_fen = pos_fen - position
                    if pos_liv < position: dist_to_liv = num_spaces + (pos_liv - position)
                    else: dist_to_liv = pos_liv - position

                    minimum_distance = min([dist_to_king, dist_to_mar, dist_to_fen, dist_to_liv])
                    position += minimum_distance
                    position = position % num_spaces

                elif "Advance token to nearest Utility" in chance_card:
                    # add current position before moving
                    space_freq[position] += 1

                    pos_elec = property_indices["Electric Company"]
                    pos_water = property_indices["Water Works"]

                    if pos_elec < position: dist_to_elec = num_spaces + (pos_elec - position)
                    else: dist_to_elec = pos_elec - position
                    if pos_water < position: dist_to_water = num_spaces + (pos_water - position)
                    else: dist_to_water = pos_water - position

                    minimum_distance = min([dist_to_elec, dist_to_water])
                    position += minimum_distance
                    position = position % num_spaces

                # Implement the effects of the Chance card
                elif "Advance to" in chance_card:
                    # add current position before moving
                    space_freq[position] += 1

                    # Move the player to the specified property
                    target_property = chance_card.split("Advance to ")[1].split(".")[0]
                    position = property_indices[target_property]

                elif "Go back to" in chance_card:
                    # add current position before moving
                    space_freq[position] += 1

                    # Move the player to the specified property
                    target_property = chance_card.split("Go back to ")[1].split(".")[0]
                    position = property_indices[target_property]

                elif "Take a trip to" in chance_card:
                    # add current position before moving
                    space_freq[position] += 1

                    # Move the player to the specified property
                    target_property = chance_card.split("Take a trip to ")[1].split(".")[0]
                    position = property_indices[target_property]

                elif "Go directly to jail" in chance_card:
                    # add current position before moving
                    space_freq[position] += 1

                    # Send the player to jail
                    in_jail = True
                    position = property_indices["Jail"]

                elif "Go back" in chance_card:
                    # add current position before moving
                    space_freq[position] += 1

                    # Send the player back by x spaces
                    position -= int(chance_card.split()[2])

                elif "Get Out of Jail Free." in chance_card:
                    get_out_jail = True

            # if rather than elif to allow for chance situation in which player moves back 3 spaces to community chest
            if property_names[position][:15] == "Community Chest":
                # Simulate drawing a Community Chest card
                community_chest_card = community_chest_cards[top_community_chest_card_idx]
                top_community_chest_card_idx = (top_community_chest_card_idx + 1) % num_community_chest_cards

                # Implement the effects of the Chance card
                if "Advance to" in community_chest_card:
                    # add current position before moving
                    space_freq[position] += 1

                    # Move the player to the specified property
                    target_property = community_chest_card.split("Advance to ")[1].split(".")[0]
                    position = property_indices[target_property]

                elif "Go directly to jail" in community_chest_card:
                    # add current position before moving
                    space_freq[position] += 1

                    # Send the player to jail
                    in_jail = True
                    position = property_indices["Jail"]

                elif "Get Out of Jail Free." in community_chest_card:
                    get_out_jail = True

            # Update the frequency of landing on the current space
            space_freq[position] += 1

# Calculate probabilities by normalizing the frequencies
space_probabilities = np.array(space_freq) / sum(space_freq)

# Update the DataFrame with the calculated probabilities
df['Probability'] = space_probabilities

# Create a bar chart with custom colors and specified x-axis order
fig = px.bar(
    df,
    x='Property',
    y='Probability',
    title="Monopoly Board Space Probabilities",
    labels={'x': 'Space', 'y': 'Probability'},
    color='Property',
    color_discrete_map=color_dict,
    category_orders={"Property": property_names}
)

#fig.update_yaxes(range=[0.02, 0.03])
fig.update_layout(showlegend=False)
fig.update_xaxes(tickangle=45, tickfont=dict(size=8))
fig.show()

In [2]:
df.head()

,Property,Color,Purchase Price,House Price,Rent,Rent Doubled,Rent 1 House,Rent 2 Houses,Rent 3 Houses,Rent 4 Houses,Rent Hotel,Probability
0,Go,lightgray,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.027404
1,Old Kent Road,brown,60.0,50.0,2.0,4.0,10.0,30.0,90.0,160.0,250.0,0.018444
2,Community Chest 1,lightgray,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.019495
3,Whitechapel Road,brown,80.0,50.0,4.0,8.0,20.0,60.0,180.0,320.0,450.0,0.020038
4,Income Tax,lightgray,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.022896
